In [97]:
import json
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np

import re
import sys
import itertools
from torch.utils.data import Dataset, DataLoader

import random
import os
import pickle
import codecs
from gensim import corpora
import gensim

In [101]:
dataset = MyDataset()
length = len(dataset)

train_dataset = dataset[:int(0.8*length)]
test_dataset = dataset[int(0.8*length):]

train_dataset = train_set(train_dataset)
test_dataset = test_set(test_dataset)